# Bronze.csv

Energia grupowana po regionie i czasie + informacje o regionie + informacje o sasiadach. LSTM, Dense, XGB

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
import xgboost as xgb
warnings.filterwarnings('ignore')

2023-09-26 01:02:12.927923: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 01:02:15.632721: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('../../data/data_world.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,1930-12-08T08:01:02.000Z,23.261,120.277,15.0,6.3,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1930-12-03T18:51:47.000Z,18.233,96.298,10.0,7.4,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1930-12-02T07:01:30.000Z,25.854,98.356,35.0,6.2,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1930-11-28T07:32:56.000Z,18.779,-106.767,15.0,6.3,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1930-11-25T19:02:53.000Z,35.050,139.129,15.0,6.9,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df[["time", "latitude", "longitude", "mag"]]

In [4]:
df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%dT%H:%M:%S.%fZ")
df

,time,latitude,longitude,mag
0,1930-12-08 08:01:02.000,23.2610,120.2770,6.3
1,1930-12-03 18:51:47.000,18.2330,96.2980,7.4
2,1930-12-02 07:01:30.000,25.8540,98.3560,6.2
3,1930-11-28 07:32:56.000,18.7790,-106.7670,6.3
4,1930-11-25 19:02:53.000,35.0500,139.1290,6.9
...,...,...,...,...
797041,2018-09-01 01:14:38.230,-30.4830,-177.9279,4.3
797042,2018-09-01 01:07:59.120,-10.7558,124.3621,4.0
797043,2018-09-01 01:00:13.810,-5.5167,147.1735,4.6
797044,2018-09-01 00:27:11.440,46.8819,155.6566,4.3


In [5]:
time_cut = dt.datetime(1973, 1, 1)
df = df[df["time"] > time_cut]
df["energy"] = 10**(1.44*df["mag"]+5.24)
df["label"] = np.where(df["mag"] > 5, 1, 0)

In [6]:
time_split = 30

In [7]:
# group the data into 30 days intervals
df["time"] = df["time"].dt.floor('d')
df["time"] = df["time"] - np.min(df["time"])
df["time"] = df["time"].dt.days // time_split
df["time"] = df["time"].astype(int)

In [8]:
geo_split = 10

In [9]:
df["latitude"] = df["latitude"] // geo_split
df["latitude"] = df["latitude"] + np.abs(np.min(df["latitude"]))

df["longitude"] = df["longitude"] // geo_split
df["longitude"] = df["longitude"] + np.abs(np.min(df["longitude"]))

In [10]:
df

,time,latitude,longitude,mag,energy,label
21801,12,7.0,34.0,4.5,5.248075e+11,0
21802,12,6.0,11.0,4.4,3.767038e+11,0
21803,12,7.0,0.0,5.1,3.837072e+12,1
21804,12,7.0,34.0,4.4,3.767038e+11,0
21805,12,6.0,34.0,4.6,7.311391e+11,0
...,...,...,...,...,...,...
797041,555,5.0,0.0,4.3,2.703958e+11,0
797042,555,7.0,30.0,4.0,1.000000e+11,0
797043,555,8.0,32.0,4.6,7.311391e+11,0
797044,555,13.0,33.0,4.3,2.703958e+11,0


In [11]:
df_agg = df.groupby(["latitude", "longitude", "time"]).agg({"energy": "sum", "label": "sum"}).reset_index()
df_agg

,latitude,longitude,time,energy,label
0,0.0,13.0,268,1.018591e+12,0
1,0.0,22.0,482,3.767038e+11,0
2,0.0,31.0,375,5.248075e+11,0
3,0.0,31.0,473,7.311391e+11,0
4,0.0,33.0,280,2.703958e+11,0
...,...,...,...,...,...
71216,17.0,30.0,487,1.018591e+12,0
71217,17.0,30.0,489,3.767038e+11,0
71218,17.0,30.0,509,7.311391e+11,0
71219,17.0,30.0,538,3.767038e+11,0


In [12]:
df_agg["label"] = np.where(df_agg["label"] > 0, 1, 0)

In [13]:
def make_ds(df_agg, block_size):
    dfs_train, dfs_val = [], []
    for i in df_agg["latitude"].unique():
        for j in df_agg["longitude"].unique():
            tmp = df_agg[(df_agg["latitude"] == i) & (df_agg["longitude"] == j)]
            if not tmp.empty:
                start = max(tmp["time"].min() - block_size, 0)
                end = min(tmp["time"].max() + block_size, df_agg["time"].max())
                tmp = tmp.set_index("time").reindex(range(start, end)).fillna(0).rename_axis('time')
                tmp["label"] = tmp["label"].shift(-1)
                tmp["latitude"] = i
                tmp["longitude"] = j
                tmp["neighbor0"] = 0
                n = int(0.8 * len(tmp))
                df_train = tmp[:n]
                df_val = tmp[n:]    
                dfs_train.append(df_train)
                dfs_val.append(df_val)
    df_final_train = pd.concat(dfs_train)
    df_final_val = pd.concat(dfs_val)
    return df_final_train, df_final_val

In [14]:
block_size = 16
df_train, df_val = make_ds(df_agg, block_size)

In [15]:
df_train

,latitude,longitude,energy,label,neighbor0
time,,,,,
252,0.0,13.0,0.000000e+00,0.0,0
253,0.0,13.0,0.000000e+00,0.0,0
254,0.0,13.0,0.000000e+00,0.0,0
255,0.0,13.0,0.000000e+00,0.0,0
256,0.0,13.0,0.000000e+00,0.0,0
...,...,...,...,...,...
488,17.0,23.0,0.000000e+00,0.0,0
489,17.0,23.0,0.000000e+00,0.0,0
490,17.0,23.0,7.311391e+11,0.0,0


In [16]:
for i in df_train["latitude"].unique():
    for j in df_train["longitude"].unique():
        if not df_train[(df_train["latitude"] == i) & (df_train["longitude"] == j)].empty:
            for x in [-1, 0, 1]:
                for y in [-1, 0, 1]:
                    if not df_train[(df_train["latitude"] == i + x) & (df_train["longitude"] == j + y)].empty and not (x == 0 and y == 0):
                        df_train.loc[(df_train["latitude"] == i) & (df_train["longitude"] == j), "neighbor0"] += df_train[(df_train["latitude"] == i + x) & (df_train["longitude"] == j + y)]["energy"]
                        df_train["neighbor0"] = df_train["neighbor0"].fillna(0)

In [17]:
dfs_train, dfs_val = [], []
for i in df_train["latitude"].unique():
    for j in df_train["longitude"].unique():
        tmp1 = df_train[(df_train["latitude"] == i) & (df_train["longitude"] == j)]
        tmp2 = df_val[(df_val["latitude"] == i) & (df_val["longitude"] == j)]
        if not tmp1.empty and not tmp2.empty:
            scaler = MinMaxScaler()
            tmp1["neighbor0"] = scaler.fit_transform(tmp1[["neighbor0"]])
            tmp2["neighbor0"] = scaler.transform(tmp2[["neighbor0"]])
            tmp1["energy0"] = scaler.fit_transform(tmp1[["energy"]])
            tmp2["energy0"] = scaler.transform(tmp2[["energy"]])
            for idx in range(1, block_size):
                tmp1["neighbor" + str(idx)] = tmp1["neighbor0"].shift(idx)
                tmp2["neighbor" + str(idx)] = tmp2["neighbor0"].shift(idx)
                tmp1["energy" + str(idx)] = tmp1["energy0"].shift(idx)
                tmp2["energy" + str(idx)] = tmp2["energy0"].shift(idx)  
            dfs_train.append(tmp1)
            dfs_val.append(tmp2)
df_final_train = pd.concat(dfs_train)
df_final_val = pd.concat(dfs_val)

In [18]:
df_final_train

,latitude,longitude,energy,label,neighbor0,energy0,neighbor1,energy1,neighbor2,energy2,...,neighbor11,energy11,neighbor12,energy12,neighbor13,energy13,neighbor14,energy14,neighbor15,energy15
time,,,,,,,,,,,,,,,,,,,,,
252,0.0,13.0,0.000000e+00,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,0.0,13.0,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,0.0,13.0,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
255,0.0,13.0,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,0.0,13.0,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,17.0,23.0,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489,17.0,23.0,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
490,17.0,23.0,7.311391e+11,0.0,0.0,0.178002,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df_final_train.dropna(inplace=True)
df_final_val.dropna(inplace=True)

In [20]:
x_train = df_final_train.drop(["label", "longitude", "latitude", "energy"], axis=1).to_numpy()
y_train = df_final_train["label"].to_numpy()
x_val = df_final_val.drop(["label", "longitude", "latitude", "energy"], axis=1).to_numpy()
y_val = df_final_val["label"].to_numpy()

In [21]:
x_train = x_train.reshape(-1, block_size, 2)
x_val = x_val.reshape(-1, block_size, 2)
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)

In [22]:
# add anoother column whith 1 - label
y_train = np.concatenate((y_train, 1 - y_train), axis=1)
y_val = np.concatenate((y_val, 1 - y_val), axis=1)

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, input_shape=(block_size, 2), return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(2),
])

2023-09-26 01:03:24.731731: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-26 01:03:24.832137: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-26 01:03:24.832272: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-26 01:03:24.838373: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-26 01:03:24.838502: I tensorflow/compile

In [24]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [25]:
model.fit(x_train, y_train, epochs=20, batch_size=512, validation_data=(x_val, y_val))

Epoch 1/20


2023-09-26 01:03:40.436367: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /home/majkel/miniconda3/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2023-09-26 01:03:40.932711: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fcf58079d50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-26 01:03:40.932783: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2060 with Max-Q Design, Compute Capability 7.5
2023-09-26 01:03:40.984714: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-26 01:03:41.527616: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the proces

329/329 [==============================] - 50s 107ms/step - loss: 0.3607 - accuracy: 0.8831 - f1_score: 0.4703 - val_loss: 0.3724 - val_accuracy: 0.8772 - val_f1_score: 0.4673
Epoch 2/20
329/329 [==============================] - 37s 113ms/step - loss: 0.3499 - accuracy: 0.8851 - f1_score: 0.4696 - val_loss: 0.3612 - val_accuracy: 0.8775 - val_f1_score: 0.4674
Epoch 3/20
329/329 [==============================] - 45s 136ms/step - loss: 0.3390 - accuracy: 0.8851 - f1_score: 0.4695 - val_loss: 0.3534 - val_accuracy: 0.8775 - val_f1_score: 0.4674
Epoch 4/20
329/329 [==============================] - 48s 147ms/step - loss: 0.3249 - accuracy: 0.8851 - f1_score: 0.4696 - val_loss: 0.3438 - val_accuracy: 0.8775 - val_f1_score: 0.4674
Epoch 5/20
329/329 [==============================] - 47s 142ms/step - loss: 0.3174 - accuracy: 0.8851 - f1_score: 0.4697 - val_loss: 0.3273 - val_accuracy: 0.8775 - val_f1_score: 0.4674
Epoch 6/20
329/329 [==============================] - 54s 166ms/step - loss:

In [26]:
x_train = df_final_train.drop(["label", "longitude", "latitude", "energy"], axis=1).to_numpy()
y_train = df_final_train["label"].to_numpy()
x_val = df_final_val.drop(["label", "longitude", "latitude", "energy"], axis=1).to_numpy()
y_val = df_final_val["label"].to_numpy()

In [27]:
x_train = x_train.reshape(-1, block_size, 2)
x_val = x_val.reshape(-1, block_size, 2)
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)

In [28]:
# add anoother column whith 1 - label
y_train = np.concatenate((y_train, 1 - y_train), axis=1)
y_val = np.concatenate((y_val, 1 - y_val), axis=1)

In [29]:
x_pos_train = df_final_train[["longitude", "latitude"]].to_numpy()
x_pos_val = df_final_val[["longitude", "latitude"]].to_numpy()

In [34]:
class MyModel(tf.keras.Model):
    def __init__(self, n_embed, vocab_size):
        super(MyModel, self).__init__()
        self.n_embed = n_embed
        self.loc_emb_x = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.loc_emb_y = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.conc = tf.keras.layers.Concatenate()
        self.dense_h = tf.keras.layers.Dense(2 * n_embed)
        self.dense_c = tf.keras.layers.Dense(2 * n_embed)
        self.lstm1 = tf.keras.layers.LSTM(n_embed * 2, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_embed * 2)
        self.dense = tf.keras.layers.Dense(n_embed, activation='relu')
        self.out = tf.keras.layers.Dense(2)
        

    def call(self, inputs):
        loc, timestep = inputs
        x = self.loc_emb_x(loc[:, 0])
        y = self.loc_emb_y(loc[:, 1])
        x_loc = self.conc([x, y])
        x_loc = tf.reshape(x_loc, [-1, 2 * self.n_embed])
        h = self.dense_h(x_loc)
        c = self.dense_c(x_loc)
        x = self.lstm1(timestep, initial_state=[c, h])
        x = self.lstm2(x, initial_state=[c, h])
        x = self.dense(x)
        return self.out(x)

In [35]:
n_embed = 256
vocab_size = 2000
model = MyModel(n_embed, vocab_size)

In [36]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [37]:
model.fit([x_pos_train, x_train], y_train, epochs=20, batch_size=512, validation_data=([x_pos_val, x_val], y_val))

Epoch 1/20
329/329 [==============================] - 85s 174ms/step - loss: 0.2893 - accuracy: 0.9003 - f1_score: 0.6484 - val_loss: 0.2614 - val_accuracy: 0.9057 - val_f1_score: 0.7305
Epoch 2/20
329/329 [==============================] - 47s 143ms/step - loss: 0.2332 - accuracy: 0.9146 - f1_score: 0.7469 - val_loss: 0.2465 - val_accuracy: 0.9064 - val_f1_score: 0.7176
Epoch 3/20
329/329 [==============================] - 64s 195ms/step - loss: 0.2194 - accuracy: 0.9180 - f1_score: 0.7569 - val_loss: 0.2336 - val_accuracy: 0.9120 - val_f1_score: 0.7611
Epoch 4/20
329/329 [==============================] - 58s 175ms/step - loss: 0.2144 - accuracy: 0.9190 - f1_score: 0.7559 - val_loss: 0.2283 - val_accuracy: 0.9116 - val_f1_score: 0.7446
Epoch 5/20
329/329 [==============================] - 55s 167ms/step - loss: 0.2115 - accuracy: 0.9196 - f1_score: 0.7571 - val_loss: 0.2252 - val_accuracy: 0.9129 - val_f1_score: 0.7473
Epoch 6/20
329/329 [==============================] - 54s 165ms/s

In [168]:
x_train = df_final_train.drop(["label", "energy"], axis=1).to_numpy()
y_train = df_final_train["label"].to_numpy()
x_val = df_final_val.drop(["label", "energy"], axis=1).to_numpy()
y_val = df_final_val["label"].to_numpy()

In [169]:
model = xgb.XGBClassifier(n_estimators=1000, max_depth=10, learning_rate=0.01, n_jobs=-1)

In [170]:
model.fit(x_train, y_train, eval_set=[(x_val, y_val)], eval_metric='logloss', early_stopping_rounds=10)

[0]	validation_0-logloss:0.68624
[1]	validation_0-logloss:0.67951
[2]	validation_0-logloss:0.67290
[3]	validation_0-logloss:0.66642
[4]	validation_0-logloss:0.66005
[5]	validation_0-logloss:0.65382
[6]	validation_0-logloss:0.64769
[7]	validation_0-logloss:0.64167
[8]	validation_0-logloss:0.63576
[9]	validation_0-logloss:0.62997
[10]	validation_0-logloss:0.62427
[11]	validation_0-logloss:0.61870
[12]	validation_0-logloss:0.61321
[13]	validation_0-logloss:0.60781
[14]	validation_0-logloss:0.60252
[15]	validation_0-logloss:0.59730
[16]	validation_0-logloss:0.59220
[17]	validation_0-logloss:0.58717
[18]	validation_0-logloss:0.58222
[19]	validation_0-logloss:0.57740
[20]	validation_0-logloss:0.57263
[21]	validation_0-logloss:0.56793
[22]	validation_0-logloss:0.56329
[23]	validation_0-logloss:0.55871
[24]	validation_0-logloss:0.55423
[25]	validation_0-logloss:0.54980
[26]	validation_0-logloss:0.54549
[27]	validation_0-logloss:0.54126
[28]	validation_0-logloss:0.53705
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [171]:
y_pred = model.predict(x_val)
accuracy_score(y_val, y_pred), f1_score(y_val, y_pred)

(0.9064726515400636, 0.5715003138731952)